<a href="https://colab.research.google.com/github/rambendavid/ucb-w207-applied-ml/blob/master/roBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.5MB 26.6MB/s 
     |████████████████████████████████| 3.3MB 48.4MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 245kB 23.9MB/s 
     |████████████████████████████████| 122kB 51.6MB/s 
     |████████████████████████████████| 245kB 42.5MB/s 


In [3]:
#!pip install -q tensorflow-addons
#!pip install -q -U tensorflow-text
#!pip install -q -U tf-models-official
#!pip install -U tfds-nightly


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [5]:
from datasets import list_datasets
datasets_list = list_datasets()

print(len(datasets_list))
print(', '.join(dataset for dataset in datasets_list))
#raw_datasets = load_dataset("qnli")

965
acronym_identification, ade_corpus_v2, adversarial_qa, aeslc, afrikaans_ner_corpus, ag_news, ai2_arc, air_dialogue, ajgt_twitter_ar, allegro_reviews, allocine, alt, amazon_polarity, amazon_reviews_multi, amazon_us_reviews, ambig_qa, amttl, anli, app_reviews, aqua_rat, aquamuse, ar_cov19, ar_res_reviews, ar_sarcasm, arabic_billion_words, arabic_pos_dialect, arabic_speech_corpus, arcd, arsentd_lev, art, arxiv_dataset, ascent_kb, aslg_pc12, asnq, asset, assin, assin2, atomic, autshumato, babi_qa, banking77, bbaw_egyptian, bbc_hindi_nli, bc2gm_corpus, best2009, bianet, bible_para, big_patent, billsum, bing_coronavirus_query_set, biomrc, blended_skill_talk, blimp, blog_authorship_corpus, bn_hate_speech, bookcorpus, bookcorpusopen, boolq, bprec, break_data, brwac, bsd_ja_en, bswac, c3, c4, cail2018, caner, capes, catalonia_independence, cawac, cbt, cc100, cc_news, ccaligned_multilingual, cdsc, cdt, cfq, chr_en, cifar10, cifar100, circa, civil_comments, clickbait_news_bg, climate_fever, c

In [6]:
from datasets import load_dataset
#dataset = load_dataset('squad')
dataset = load_dataset('glue', 'qnli')


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
})

In [8]:
dataset['train'][0]

{'idx': 0,
 'label': 1,
 'question': 'When did the third Digimon series begin?',
 'sentence': 'Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.'}

In [9]:
from transformers import TFRobertaForQuestionAnswering

In [10]:
model_type = 'thatdramebaazguy/roberta-base-squad'
model = TFRobertaForQuestionAnswering.from_pretrained(model_type)

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

All the layers of TFRobertaForQuestionAnswering were initialized from the model checkpoint at thatdramebaazguy/roberta-base-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.


In [11]:
# convert text to tokens:

from transformers import AutoTokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_type)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [14]:
print(dataset)
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['idx', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
})
DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'question', 'sentence'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'question', 'sentence'],
        num_rows: 5463
    })
})


In [15]:
tokenized_datasets['train']

Dataset({
    features: ['attention_mask', 'idx', 'input_ids', 'label', 'question', 'sentence'],
    num_rows: 104743
})

In [16]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [17]:
print(small_train_dataset['attention_mask'][0])
print(small_train_dataset['idx'][0])
print(small_train_dataset['input_ids'][0])
print(small_train_dataset['label'][0])
print(small_train_dataset['question'][0])
print(small_train_dataset['sentence'][0])
small_train_dataset


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Dataset({
    features: ['attention_mask', 'idx', 'input_ids', 'label', 'question', 'sentence'],
    num_rows: 1000
})

In [18]:
tf_train_dataset = small_train_dataset.remove_columns(["sentence"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["sentence"]).with_format("tensorflow")

tf_train_dataset

Dataset({
    features: ['attention_mask', 'idx', 'input_ids', 'label', 'question'],
    num_rows: 1000
})

In [19]:
train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [20]:
print(train_tf_dataset)
train_features

<BatchDataset shapes: ({input_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>


{'attention_mask': <tf.Tensor: shape=(1000, 512), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 'input_ids': <tf.Tensor: shape=(1000, 512), dtype=int64, numpy=
 array([[    0, 40860,     5, ...,     1,     1,     1],
        [    0, 14229,    42, ...,     1,     1,     1],
        [    0,   133, 38316, ...,     1,     1,     1],
        ...,
        [    0,   133,  2946, ...,     1,     1,     1],
        [    0,   243,    16, ...,     1,     1,     1],
        [    0,   713,    16, ...,     1,     1,     1]])>}

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

In [22]:
model.summary()

Model: "tf_roberta_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 124,056,578
Trainable params: 124,056,578
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [23]:
model.fit(train_tf_dataset, validation_data=train_tf_dataset, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [59]:
#import tensorflow_addons as tfa
import tensorflow as tf
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [58]:
metric_name = "matthews_correlation"
batch_size = 16

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tfa.metrics.MatthewsCorrelationCoefficient(num_classes=3),
)

In [13]:
from transformers import pipeline

In [21]:
nlp = pipeline('question-answering',model=model, tokenizer=tokenizer)

NameError: ignored